In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook()

# CS187
## Lab 1-5 – Scaling up: Torchtext and PyTorch

The pipeline for NLP applications based on supervised machine learning involves several standard components:

1. Loading of annotated textual corpora.
2. Tokenization and normalization of the text.
3. Distributing instances into subcorpora, for instance, training, development, and test corpora.
4. Training of models on training data.
5. Evaluation of the models on test data.

Rather than recapitulate all of these component tasks for each application, standard packages have been developed to facilitate them. In order to facilitate your own experimentation, it's time to make use of some of these packages to scale up your ability to build and test models. That is the subject of this lab.

Torchtext datasets provide a uniform system for establishing dataset objects that contain multiple examples, each of which may have multiple named fields. These fields themselves have specifications that tell whether the data in that field is sequential (like text sequences) or simple (like class labels); whether and how to preprocess, tokenize, or postprocess the data; and many other properties. Dataset objects can be easily split into parts (training and test, for instance), or turned into a sequence of small batches for processing by models.

This lab provides an introduction to using `torchtext` and PyTorch in preparation for its appearance in later labs and project segments.

New bits of Python used for the first time in the _solution set_ for this lab, and which you may therefore find useful:

* [`torch.Tensor.backward`](https://pytorch.org/docs/stable/autograd.html#torch.Tensor.backward)


In [1]:
import os
import copy
import math
import matplotlib.pyplot as plt
import numpy as np
import re
import sys
import torch
import torch.distributions as ds
import torch.nn as nn
import torch.nn.functional as F
import torchtext as tt

from torch import optim

# Otter grader which we use for grading does not support
# !command, so we need to use shell(command) instead
# to run shell commands
def shell(str):
    file = os.popen(str)
    result = file.read()
    print (result)
    if file.close () is not None:
        print ('failed')

%matplotlib inline
plt.style.use('tableau-colorblind10')

# Random seed
random_seed = 1234
## GPU check
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Part 1: Manipulating text corpora with `torchtext`

You'll use `torchtext` to load the _Green Eggs and Ham_ (GEaH) dataset.

We start with reading in the data and performing some ad hoc cleaning (removing comment lines and blank lines).

In [2]:
def strip_file(text):
    """strip #comments and empty lines from a string"""
    result = ""
    for line in text.split("\n"):
        line = line.strip()              # trim whitespace
        line = re.sub('#.*$', '', line)  # trim comments
        if line != '':                   # drop blank lines
            result += line + '\n'
    return result

# Read the GEaH data and write out a corresponding TSV file
shell('wget -nv -N -P data "https://github.com/nlp-course/data/raw/master/Seuss/seuss - 1960 - green eggs and ham.txt"')
with open('data/seuss - 1960 - green eggs and ham.txt', 'r') as fin:
    with open('data/geah.tsv', 'w') as fout:
        fout.write(strip_file(fin.read()))

## Constructing training and test datasets

Take a look at the file `geah.tsv`, which we've just processed and placed into the sibling `data` folder. 

In [3]:
shell('head "data/geah.tsv"')

Notice the structure of this corpus. Each line contains a sentence from the book, preceded by a label that provides the speaker of that sentence. The speaker and sentence are separated by a tab character. The data is thus set up properly for a [`torchtext.data.TabularDataset`](https://pytorch.org/text/data.html#torchtext.data.TabularDataset) using its `"TSV"` (tab-separated values) format.

In order to establish a `torchtext.data.TabularDataset` object for dealing with the GEaH dataset, you'll first need to establish the two fields (via [`torchtext.data.Field`](https://pytorch.org/text/data.html#field)), one for the label (the speaker) and one for the text, which you should call `LABEL` and `TEXT`, respectively.

When setting up a field with `torchtext.data.Field`, you'll want to consider whether you want to further specify the values for the various keyword arguments listed [here](https://torchtext.readthedocs.io/en/latest/data.html#field), or to leave the default values.

With respect to the tokenization of the text field, you should use the `torchtext` "basic_english" tokenizer introduced in lab 1-1 for the text field and lowercase all tokens.
<!--
BEGIN QUESTION
name: fields_setup
-->

In [4]:
#TODO
LABEL = (
    ...
)
    
TEXT = (
    ...
)

In [ ]:
grader.check("fields_setup")

Now, you can set up the dataset using `torchtext.data.TabularDataSet`. It should look for the TSV data in the file `data/geah/tsv`, and should use the names `label` and `field` for the two fields in the tabular data.
<!--
BEGIN QUESTION
name: dataset_setup
-->

In [6]:
#TODO
geah = (
    ...
)

All [`torchtext.data.Dataset`](https://pytorch.org/text/data.html#torchtext.data.Dataset) objects have a [`split`](https://pytorch.org/text/data.html#torchtext.data.Dataset.split) method that splits the dataset into two or three pieces, for instance, to have a separate training and test set. Use the `split` method to generate a 70%/30% split of the GEaH corpus into two subsets called `train` and `test`.
<!--
BEGIN QUESTION
name: dataset_split
-->

In [7]:
#TODO
train, test = geah.split() # Solution

Fields can have a ssociated with them a _vocabulary_ consisiting of all of the possible values that are used in that field in a particular dataset. The vocabulary establishes the kind of indexing scheme between types and indices that we explored in lab 1-1. We will use the training corpus to establish vocabularies for the two fields `LABEL` and `TEXT` using the [`build_vocab`](https://pytorch.org/text/data.html#torchtext.data.Field.build_vocab) method.

In [8]:
LABEL.build_vocab(train)
TEXT.build_vocab(train)

The `TEXT` and `LABEL` fields, objects of class [`torchtext.data.Field`](https://pytorch.org/text/data.html#field), now have vocabularies associated with them, accessible in their respective `vocab` fields. How many elements are there in these vocabularies? You can use the `len` function to find out.
<!--
BEGIN QUESTION
name: vocab_sizes
-->

In [9]:
#TODO
label_vocab_size = len(LABEL.vocab) # Solution
text_vocab_size = len(TEXT.vocab)   # Solution

In [ ]:
grader.check("vocab_sizes")

In [12]:
print(f"label vocabulary size is {label_vocab_size}\n"
      f"text vocabulary size is {text_vocab_size}")

Why are there three elements in the `LABEL` vocabulary, given that there are only two speakers, Guy and Sam? We can find out by examining the `LABEL` vocabulary more closely. `Field` vocabulary objects have an especially useful `stoi` data field, whose value is a dictionary that maps the elements of the vocabulary to integer index representations of the elements. Let's take a look.

In [13]:
LABEL.vocab.stoi

<!-- BEGIN QUESTION -->

**Question:** What is the unexpected third element in the label vocabulary? Why do you think it's there?
<!--
BEGIN QUESTION
name: open_response_third_element
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



> As described in more detail in [the `torchtext` documentation](https://torchtext.readthedocs.io/en/latest/vocab.html#torchtext.vocab.Vocab), there's also an `itos` data field for conversion of vocabulary items from the index representation to the original values and a `freqs` data field that keeps a frequency distribution for the items in the vocabulary as a `collections.Counter` object.

## Operations over datasets

We now have training and test datasets. You can experiment with the kinds of operations you'll need to do to implement models like Naive Bayes or logistic regression.

For instance, you might need to iterate over the different class labels (the vocabulary of the `LABEL` field) or the word types (the vocabulary of the `TEXT` field). Define a function that iterates over the vocabulary of a field and prints each one out like this:
```
>>> print_vocab(LABEL)
<unk>
GUY
SAM
```
<!--
BEGIN QUESTION
name: print_vocab
-->

In [14]:
#TODO
def print_vocab(field):
    ...

In [ ]:
grader.check("print_vocab")

 We can use the `print_vocab` function to print out the different class labels in the `LABEL` field.

In [16]:
print_vocab(LABEL)

Other simple calculations that will be useful in implementing the various models:

1. Counting how many instances there are in a dataset.
2. Counting how many instances of a certain class there are in a data set.
3. Counting how many tokens of a certain type there are in the text of an instance.

Let's write functions for these. They'll come in handy in the first problem set.
<!--
BEGIN QUESTION
name: count_instances
-->

In [17]:
#TODO - 1. Counting how many instances there are in a dataset.
def count_instances(dataset):
    ...

In [ ]:
grader.check("count_instances")

In [19]:
#TODO - 2. Counting how many instances of a certain class there are in a data set.
def count_instances_class(dataset, label):
    ...

In [ ]:
grader.check("count_instances_class")

In [21]:
#TODO - 3. Counting how many tokens of a certain type there are in the text of an instance.
def count_tokens_instance(instance, tokentype):
    ...

In [ ]:
grader.check("count_tokens_instances")

# Part 2: Training and testing with PyTorch

Past labs have shown that all of the detail about 

* establishing models and their parameters,
* using them to calculate the outputs for some inputs, 
* training them to optimize the parameters via stochastic gradient descent, and 
* evaluating them by testing on held-out data

is tedious to manage. Fortunately, it is also so formulaic, at least for a certain class of models, that general tools can be deployed to manage the process. In the remainder of this lab, you'll use one such tool, PyTorch. For simplicity, rather than a natural-language task, you'll be training a model to fit a curve; it has an especially simple structure: one scalar input and one scalar output.

## Generating training and test data

We start by generating some training and test data. The data is generated as a noisy sine function, calculated by the function `sinusoid`.

In [23]:
def sinusoid(x, amplitude=1., phase=0., frequency=1., noise=0.):
    """Returns the values on input(s) `x` of a sinusoid determined by `amplitude`, 
       `phase`, and angular `frequency`, with some added normal noise with variance 
       given by `noise`."""
    normal_noise = ds.normal.Normal(torch.tensor([0.0]), torch.tensor([noise]))
    noise_sample = torch.tensor([normal_noise.sample() for i in range(len(x))])
    y = amplitude * torch.sin(x * frequency + phase) + noise_sample
    return y

We can generate data for training and testing by sampling this function.

In [24]:
def sample_input(func, count, bound, **kwargs):
    """Returns `count` samples of x-y pairs of function `func`, with the x 
       values sampled uniformly between +/-`bound`. The `kwargs` are passed
       on to `func`."""
    input_unif = ds.uniform.Uniform(-bound, +bound)
    x = input_unif.sample(torch.Size([count]))
    y = func(x, **kwargs)
    return x, y

To give a sense of what a data sample looks like, we plot a sample of 100 points.

In [25]:
def plot_sample(data):
    """Plots `data` given as a single pair of inmputs and outputs."""
    X, Y = data
    plt.plot(X.numpy(), Y.numpy(), '.')
    plt.xlabel('Input')
    plt.ylabel('Output')
    # we cannot use plt.show() because otter-grader does not support it

In [26]:
plot_sample(sample_input(sinusoid, 100, 5, noise=0.1))

## Specifying a feed-forward neural network

<img src="https://github.com/nlp-course/data/raw/feature/ffnnfig/Resources/ffnn-example.png" width=33% align=right />
The model that we will train to predict the output of this function based on a sample will consist of a series of sublayers as depicted in the figure at right. At the bottom of the figure, we start with $\vect{x}$, the scalar input (of dimensionality $1$ as shown in the "shape" designation). The first layer is a perceptron layer, with a linear sublayer (with weights $\vect{U}$) followed by a sigmoid sublayer. Since $U$ is of dimenionality $1 \times D$, the output is a vector of dimensionality $D$. (We refer to $D$ as the hidden dimension.) Then comes another perceptron layer with output of dimensionality $D$. Finally, a single linear layer reduces the dimensionality back to the predicted scalar output $\tilde{y}$ of dimensionality $1$. The loss is calculated as the mean square error of $\tilde{y}$ relative to $y$. (In this case, the mean is irrelevant, since $y$ is a scalar.)

We define a class `FFNN`, which inherits from the `nn.Module` class, PyTorch's class for neural network models. It takes an argument `hidden_dim` which is the size of the hidden layers, $D$ in the figure.

The parameters of this model – the values that will be adjusted to minimize the loss – are the elements of the tensors $\vect{U}$, $\vect{V}$, and $\vect{W}$. Those parameters are created and tracked when the corresponding sublayers are created using `nn.Linear`. That's the wonder of using PyTorch – so much happens under the hood.

In [27]:
class FFNN(nn.Module):
    def __init__(self, hidden_dim, init_low=-2, init_high=2):
        super().__init__()
        # dimensionality of hidden layers
        self.hidden_dim = hidden_dim
        # the sublayers, two perceptrons and a final linear layer
        self.sublayer1 = nn.Linear(1, hidden_dim) 
        self.sublayer2 = nn.Sigmoid() 
        self.sublayer3 = nn.Linear(hidden_dim, hidden_dim)
        self.sublayer4 = nn.Sigmoid() 
        self.sublayer5 = nn.Linear(hidden_dim, 1)
        
        # initialize parameters
        torch.manual_seed(random_seed)
        for p in self.parameters():
            p.data.uniform_(init_low, init_high)
        # save a copy of the parameters to allow resetting
        self.init_state = copy.deepcopy(self.state_dict())

    # Resetting state: If you want to rerun a model, say with a different
    # training regime, you can reset the model's parameter state using
    #    model.reset_state()
    # before retraining, e.g., 
    #    train_model(model, criterion, optim, train_data, n_epochs=50)
    def reset_state(self):
        self.load_state_dict(copy.deepcopy(self.init_state))
        
    def forward(self, x):
        # first perceptron layer
        z = self.sublayer2(self.sublayer1(x))
        # second perceptron layer
        z_prime = self.sublayer4(self.sublayer3(z))
        # final linear layer
        return self.sublayer5(z_prime)

We can build a model by instantiating the `FFNN` class. We'll do so with a hidden dimension of 4, being careful to move the model with its parameters to the device we're using for calculations (a GPU if one is available, as on Google Colab).

In [28]:
HIDDEN_DIMENSION = 4
model = FFNN(HIDDEN_DIMENSION).to(device)

We specify the criterion to be optimized as the mean square error loss function provided by PyTorch.

In [29]:
criterion = nn.MSELoss(reduction='mean') 

## Evaluating data according to a model

To evaluate how well the model performs on some test data, we run the model forward on the $x$ values and compute the loss relative to the $y$ values. We define a function `eval_model` to carry out this calculation.

In [30]:
def eval_model(model, criterion, data):
    """Applies the `model` to the x values in the `data` and returns the
       loss relative to the y values in the data along with the predicted 
       y values."""
    model.eval()                          # turn on evaluation mode
    with torch.no_grad():                 # turn off propagating gradients
        X, Y = data                       # extract x and y values
        X = X.view(-1, 1).to(device)      # convert x and y to column vectors
        Y = Y.view(-1, 1).to(device)      # ...and move them to the device
        predictions = model(X)            # calculate the predicted y values
        loss = criterion(predictions, Y)  # see how far off they are
    return loss.item(), predictions

All that remains is training the model. We'll use one of PyTorch's built in optimizers, the `Adam` optimizer. We set a few parameters for the training process: the learning rate, the number of "epochs" (passes through the training data) to perform, and the number of examples to train on at a time (the "batch size").

In [31]:
## Parameters of the training regimen
LEARNING_RATE = 0.003
NUMBER_EPOCHS = 25
BATCH_SIZE = 20

## Choices for optimizers
# Stochastic Gradient Descent (SGD) optimizer
# optim = torch.optim.SGD(model.parameters(), lr = learning_rate)
# The Adam optimizer, as described in the paper:
# Kingma and Ba. 2014. Adam: A Method for Stochastic Optimization.
# [https://arxiv.org/abs/1412.6980]
optim = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

## Training the parameters of a model

Finally, we get to the function to train the parameters of the model so as to best fit the predictions to the actual values. We've provided the code, except for a few lines that you'll need to provide (marked `#TODO`), making use of some of the tools defined above. Those lines, which form the heart of the computation, calculate "forwards" to get the output predictions for the inputs, calculate the loss for those predictions, and calculate "backwards" the gradients of the loss for each of the parameters of the model. This sets up the optimizer to take a step of updating the parameters, making use of the calculated gradients to determine the direction to step. The saved gradients can then be zeroed and the process repeated.

> Note: The code we're asking you to write is *tiny*. If you find yourself writing more than a short line of code per `#TODO`, you're missing something.

In [32]:
def train_model(model, criterion, optimizer, data,
                n_epochs=NUMBER_EPOCHS, batch_size=BATCH_SIZE):
    """Optimizes the parameters of the `model` by minimizing the `criterion`
       on the training `data`, using the `optimizer` algorithm for updates."""    
    model.train()                     # Turn on training mode

    X, Y = data
    trainX_len = len(X)
    
    for epoch in range(n_epochs):
        loss_per_epoch = 0.
        for batch_i in range(int(trainX_len/batch_size)):
            optimizer.zero_grad()     # new batch; zero the gradients of the parameters
            
            # Input tensors and their corresponding output values for this batch
            batch_X = (X[batch_i * batch_size
                         : (batch_i+1) * batch_size] # extract examples in batch
                       .view(-1, 1)                  # reshape to column vector
                       .to(device)                   # move to device
                      )
            batch_Y = (Y[batch_i * batch_size 
                         : (batch_i+1) * batch_size]
                       .view(-1, 1)
                       .to(device)
                      )
            
            #TODO: Calculate predictions for the x values in this batch
            predictions = model(batch_X)               #Solution
            
            #TODO: Calculate the loss for the predictions
            loss = criterion(predictions, batch_Y)     #Solution
            
            #TODO: Perform backpropagation to update all of the parameters
            loss.backward()                            #Solution
            
            optimizer.step()
            
            loss_per_epoch += loss.item()
        
        print(f"Epoch: {epoch+1:3d}  Total loss: {loss_per_epoch:8.3f}")

## Putting it all together

Let's try it out. We start by generating some training and test data. The training data will be 10,000 samples of a noisy sinusoid. The test data, 100 samples from the same sinusoid, will be noise-free, so we can see how close the predictions are to noise-free outputs.

In [33]:
train_data = sample_input(sinusoid, 10000, 5., frequency=1.5, noise=0.05)
test_data = sample_input(sinusoid, 100, 5., frequency=1.5)

plot_sample(train_data)

We train the model.

In [34]:
model.reset_state()
train_model(model, criterion, optim, train_data)

...and test the trained model by evaluating it on the the test data.

In [35]:
loss, predictions = eval_model(model, criterion, test_data)

We can see how well the model works by plotting the test data (circles) along with the predicted values (crosses).

In [37]:
def visualize_predictions(data, predictions):
    X, Y = data
    
    # Plot the actual output values
    plt.plot(X.cpu().numpy(), Y.cpu().numpy(), '.', label = 'Target Values')
    
    # Plot the predicted output values
    predictions = predictions.view(-1, 1)
    plt.plot(X.cpu().numpy(), predictions.cpu().numpy(), 'x', label = 'Predictions')
    
    plt.xlabel('Input')
    plt.ylabel('Output')
    plt.legend()
    # we cannot use plt.show() because otter-grader does not support it

In [38]:
# Visualize the predictions
visualize_predictions(test_data, predictions)

## Trying different models

Now that we have the infrastructure, try experimenting with different models. Here are a few things you might play with. (No need to try them all.) What happens if you change the hidden dimension, increasing it to 8 or decreasing it to 2? What happens if you drop the middle layer? What about no middle layer but a much higher hidden dimension size? Does running for more epochs improve performance? Does the SGD optimizer work better or worse than the Adam optimizer?

<!-- BEGIN QUESTION -->

**Question:** What conclusions have you drawn from your experimentation?
<!--
BEGIN QUESTION
name: open_response_testing_models
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



# End of lab 1-5

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()